In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

In [2]:
stocks = pd.read_csv('C:/Users/pconn/OneDrive/Desktop/STONKS/Projects/constituents.csv')
stocks.head()

,Symbol,Name,Sector
0,MMM,3M Company,Industrials
1,AOS,A.O. Smith Corp,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie Inc.,Health Care
4,ABMD,ABIOMED Inc,Health Care


In [3]:
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token=Tpk_059b97af715d417d9f49f50b51b1c448'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2156550866447,
 'week52high': 143.12,
 'week52low': 56.24,
 'week52change': 0.629252067953342,
 'sharesOutstanding': 17436935104,
 'float': 0,
 'avg10Volume': 99633072,
 'avg30Volume': 113093751,
 'day200MovingAvg': 121.13,
 'day50MovingAvg': 131.3,
 'employees': 0,
 'ttmEPS': 3.34,
 'ttmDividendRate': 0.8164804771342652,
 'dividendYield': 0.006476739810980853,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-26',
 'nextEarningsDate': '0',
 'peRatio': 38.72122608049984,
 'beta': 1.1731412490606743,
 'maxChangePercent': 48.59265395341569,
 'year5ChangePercent': 4.842247771256537,
 'year2ChangePercent': 2.42118982785912,
 'year1ChangePercent': 0.655178531099173,
 'ytdChangePercent': -0.04210727537021527,
 'month6ChangePercent': 0.3319263146390997,
 'month3ChangePercent': 0.07235849234776529,
 'month1ChangePercent': -0.005454509910049221,
 'day30ChangePercent': -0.012550461065720895,
 'day5ChangePercent': -0.03752511485619313}

In [4]:
pe_ratio = data['peRatio']
pe_ratio

38.72122608049984

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Symbol'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [7]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token=Tpk_059b97af715d417d9f49f50b51b1c448'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,MMM,171.87,19.43,N/A
1,AOS,59.24,30.23,N/A
2,ABT,112.40,59.30,N/A
3,ABBV,113.18,24.17,N/A
4,ABMD,338.46,73.60,N/A
...,...,...,...,...
500,YUM,110.43,31.03,N/A
501,ZBRA,426.42,47.03,N/A
502,ZBH,167.31,-225.75,N/A
503,ZION,50.23,21.33,N/A


In [8]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

In [9]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,41.59,2.58,N/A
1,NLOK,20.53,4,N/A
2,AIV,4.97,4.82,N/A
3,BIO,613.69,5.27,N/A
4,UNM,26.16,5.60,N/A
5,AFL,47.47,7.60,N/A
6,EBAY,57.97,7.70,N/A
7,ALL,112.80,7.93,N/A
8,CPB,48.15,8.07,N/A
9,KIM,17.15,8.36,N/A


In [10]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

In [11]:
portfolio_input()

Enter the value of your portfolio:100000


In [12]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,41.59,2.58,48
1,NLOK,20.53,4,97
2,AIV,4.97,4.82,402
3,BIO,613.69,5.27,3
4,UNM,26.16,5.60,76
5,AFL,47.47,7.60,42
6,EBAY,57.97,7.70,34
7,ALL,112.80,7.93,17
8,CPB,48.15,8.07,41
9,KIM,17.15,8.36,116


In [13]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token=Tpk_059b97af715d417d9f49f50b51b1c448'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [14]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token=Tpk_059b97af715d417d9f49f50b51b1c448'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

In [15]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,172.51,N/A,19.60,N/A,8.20,N/A,3.04,N/A,13.816078,N/A,7.141627,N/A,N/A
1,AOS,57.35,N/A,29.58,N/A,5.35,N/A,3.41,N/A,18.073995,N/A,8.236581,N/A,N/A
2,ABT,113.50,N/A,60.99,N/A,6.49,N/A,6.36,N/A,32.184052,N/A,11.284928,N/A,N/A
3,ABBV,111.68,N/A,25.15,N/A,13.34,N/A,4.90,N/A,16.806752,N/A,10.165162,N/A,N/A
4,ABMD,340.92,N/A,75.20,N/A,13.00,N/A,19.35,N/A,56.831049,N/A,21.671023,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.39,N/A,32.03,N/A,-4.21,N/A,5.87,N/A,23.666984,N/A,10.003390,N/A,N/A
501,ZBRA,421.74,N/A,48.99,N/A,11.70,N/A,5.26,N/A,31.752446,N/A,12.046423,N/A,N/A
502,ZBH,161.04,N/A,-222.25,N/A,2.90,N/A,4.80,N/A,26.432915,N/A,8.110235,N/A,N/A
503,ZION,50.65,N/A,21.88,N/A,1.16,N/A,2.72,N/A,11.553610,N/A,3.139818,N/A,N/A


In [16]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
24,GOOG,1807.460,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
43,AON,208.600,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
65,BRK.B,234.850,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
95,CTL,12.000,N/A,9.94,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
145,DISCK,32.910,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
158,ETFC,50.390,N/A,14.30,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
175,EVRG,56.500,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
189,FRC,161.320,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
201,FOX,30.620,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
325,MYL,16.204,N/A,32.20,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [17]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

In [18]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [19]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.407921
1      0.603960
2      0.881188
3      0.502970
4      0.920792
         ...   
500    0.659406
501    0.827723
502    0.025743
503    0.449505
504    0.815842
Name: PE Percentile, Length: 505, dtype: object
0      0.744554
1      0.616832
2      0.671287
3      0.867327
4      0.855446
         ...   
500    0.043564
501    0.837624
502    0.396040
503    0.113861
504    0.930693
Name: PB Percentile, Length: 505, dtype: object
0      0.483168
1      0.530693
2      0.766337
3      0.689109
4      0.980198
         ...   
500    0.746535
501    0.716832
502    0.681188
503    0.431683
504    0.936634
Name: PS Percentile, Length: 505, dtype: object
0      0.392079
1      0.538614
2      0.841584
3      0.506931
4      0.920792
         ...   
500    0.708911
501    0.835644
502    0.776238
503    0.273267
504    0.843564
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0      0.439604
1      0.526733
2      0.714851
3      0.665347
4      0.948515
         ...   
5

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,172.51,N/A,19.60,0.407921,8.20,0.744554,3.04,0.483168,13.816078,0.392079,7.141627,0.439604,N/A
1,AOS,57.35,N/A,29.58,0.603960,5.35,0.616832,3.41,0.530693,18.073995,0.538614,8.236581,0.526733,N/A
2,ABT,113.50,N/A,60.99,0.881188,6.49,0.671287,6.36,0.766337,32.184052,0.841584,11.284928,0.714851,N/A
3,ABBV,111.68,N/A,25.15,0.502970,13.34,0.867327,4.90,0.689109,16.806752,0.506931,10.165162,0.665347,N/A
4,ABMD,340.92,N/A,75.20,0.920792,13.00,0.855446,19.35,0.980198,56.831049,0.920792,21.671023,0.948515,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.39,N/A,32.03,0.659406,-4.21,0.043564,5.87,0.746535,23.666984,0.708911,10.003390,0.659406,N/A
501,ZBRA,421.74,N/A,48.99,0.827723,11.70,0.837624,5.26,0.716832,31.752446,0.835644,12.046423,0.750495,N/A
502,ZBH,161.04,N/A,-222.25,0.025743,2.90,0.396040,4.80,0.681188,26.432915,0.776238,8.110235,0.510891,N/A
503,ZION,50.65,N/A,21.88,0.449505,1.16,0.113861,2.72,0.431683,11.553610,0.273267,3.139818,0.156436,N/A


In [20]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,172.51,N/A,19.60,0.407921,8.20,0.744554,3.04,0.483168,13.816078,0.392079,7.141627,0.439604,0.493465
1,AOS,57.35,N/A,29.58,0.603960,5.35,0.616832,3.41,0.530693,18.073995,0.538614,8.236581,0.526733,0.563366
2,ABT,113.50,N/A,60.99,0.881188,6.49,0.671287,6.36,0.766337,32.184052,0.841584,11.284928,0.714851,0.775050
3,ABBV,111.68,N/A,25.15,0.502970,13.34,0.867327,4.90,0.689109,16.806752,0.506931,10.165162,0.665347,0.646337
4,ABMD,340.92,N/A,75.20,0.920792,13.00,0.855446,19.35,0.980198,56.831049,0.920792,21.671023,0.948515,0.925149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.39,N/A,32.03,0.659406,-4.21,0.043564,5.87,0.746535,23.666984,0.708911,10.003390,0.659406,0.563564
501,ZBRA,421.74,N/A,48.99,0.827723,11.70,0.837624,5.26,0.716832,31.752446,0.835644,12.046423,0.750495,0.793663
502,ZBH,161.04,N/A,-222.25,0.025743,2.90,0.396040,4.80,0.681188,26.432915,0.776238,8.110235,0.510891,0.478020
503,ZION,50.65,N/A,21.88,0.449505,1.16,0.113861,2.72,0.431683,11.553610,0.273267,3.139818,0.156436,0.284950


In [21]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)


In [22]:
portfolio_input()

Enter the value of your portfolio:100000


In [23]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

C:\Users\pconn\anaconda3\lib\site-packages\pandas\core\indexing.py:1719: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,F,10.05,199,-241.91,0.023762,1.1600,0.113861,0.3049,0.011881,3.905390,0.039604,2.002671,0.091089,0.056040
1,AIG,42.50,47,-7.31,0.112871,0.5789,0.057426,0.7850,0.071287,4.764968,0.045545,0.790269,0.015842,0.060594
2,AAL,15.90,125,-1.18,0.154455,-1.7600,0.047525,0.3932,0.023762,-6.173441,0.029703,1.361668,0.053465,0.061782
3,UNM,25.38,78,5.50,0.196040,0.4932,0.053465,0.4451,0.033663,2.811366,0.035644,0.440787,0.003960,0.064554
4,MET,52.32,38,8.59,0.207921,0.6362,0.061386,0.7202,0.061386,4.880201,0.049505,0.714751,0.011881,0.078416
5,BA,214.41,9,-26.60,0.075248,-10.0700,0.033663,1.9000,0.306931,-29.142216,0.017822,-199.158639,0.001980,0.087129
6,ALL,113.00,17,7.87,0.200000,1.3000,0.152475,0.7539,0.065347,2.497288,0.033663,0.731679,0.013861,0.093069
7,FTI,11.92,167,-0.97,0.156436,1.2800,0.139604,0.4125,0.029703,3.601854,0.037624,2.330846,0.110891,0.094851
8,ABC,109.20,18,-6.70,0.114851,-21.7000,0.019802,0.1156,0.003960,8.748575,0.130693,3.992902,0.223762,0.098614
9,HIG,52.63,38,10.86,0.232673,1.0700,0.099010,0.9340,0.099010,4.410309,0.041584,0.881504,0.021782,0.098812


In [24]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

In [25]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [26]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [27]:
writer.save()